#### Data Augumentation

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

def generate_augmented_images(input_image_path, output_folder_path, num_augmentations=10):
    if not os.path.exists(output_folder_path):
        os.makedirs(output_folder_path)
    else:
        if len(os.listdir(output_folder_path)) > 0:
            pass

    image = tf.keras.preprocessing.image.load_img(input_image_path, target_size=(224, 224))
    image_array = tf.keras.preprocessing.image.img_to_array(image)
    image_array = image_array / 255.0

    datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest',
    )


    augmented_images = datagen.flow(image_array[None, :, :, :], batch_size=1)

    for i, batch in enumerate(augmented_images):
        augmented_image = batch[0]
        output_image_path = os.path.join(output_folder_path, f"{i+1}.jpg")
        tf.keras.preprocessing.image.array_to_img(augmented_image * 255.0).save(output_image_path)

        if i == num_augmentations - 1:
            break

    print(f"Generated {num_augmentations} augmented images and saved them to '{output_folder_path}'")

input_image_path = "/content/drive/MyDrive/AI_and_ML_Face_Dataset/Keerthi/keerthi.jpg"
output_folder_path = "/content/drive/MyDrive/AI_and_ML_Face_Dataset/Keerthi"
generate_augmented_images(input_image_path, output_folder_path, num_augmentations=15)


### Face Recognition

#### Generating Training and Testing Data

In [ ]:
TrainingImagePath='/content/drive/MyDrive/AI_and_ML_Face_Dataset'

from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
        shear_range=0.1,
        zoom_range=0.1,
        horizontal_flip=True)

test_datagen = ImageDataGenerator()

training_set = train_datagen.flow_from_directory(
        TrainingImagePath,
        target_size=(64, 64),
        batch_size=32,
        class_mode='categorical')

test_set = test_datagen.flow_from_directory(
        TrainingImagePath,
        target_size=(64, 64),
        batch_size=32,
        class_mode='categorical')

test_set.class_indices

Found 96 images belonging to 6 classes.
Found 96 images belonging to 6 classes.


{'Ambi': 0, 'Dharma': 1, 'Hari': 2, 'Keerthi': 3, 'Pazham': 4, 'Sanjay': 5}

#### Deciding Output Neuron

In [ ]:
TrainClasses=training_set.class_indices

ResultMap={}
for faceValue,faceName in zip(TrainClasses.values(),TrainClasses.keys()):
    ResultMap[faceValue]=faceName

import pickle
with open("ResultsMap.pkl", 'wb') as fileWriteStream:
    pickle.dump(ResultMap, fileWriteStream)

print("Mapping of Face and its ID",ResultMap)

OutputNeurons=len(ResultMap)
print('\n The Number of output neurons: ', OutputNeurons)

Mapping of Face and its ID {0: 'Ambi', 1: 'Dharma', 2: 'Hari', 3: 'Keerthi', 4: 'Pazham', 5: 'Sanjay'}

 The Number of output neurons:  6


#### Model Creation

In [ ]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPool2D
from keras.layers import Flatten
from keras.layers import Dense

classifier= Sequential()

classifier.add(Convolution2D(32, kernel_size=(5, 5), strides=(1, 1), input_shape=(64,64,3), activation='relu'))
classifier.add(MaxPool2D(pool_size=(2,2)))
classifier.add(Convolution2D(64, kernel_size=(5, 5), strides=(1, 1), activation='relu'))
classifier.add(MaxPool2D(pool_size=(2,2)))
classifier.add(Flatten())
classifier.add(Dense(64, activation='relu'))
classifier.add(Dense(OutputNeurons, activation='softmax'))

classifier.compile(loss='categorical_crossentropy', optimizer = 'adam', metrics=["accuracy"])

classifier.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 60, 60, 32)        2432      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 30, 30, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 26, 26, 64)        51264     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 13, 13, 64)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 10816)             0         
                                                                 
 dense (Dense)               (None, 64)                6

#### Model Fitting

In [ ]:
classifier.fit_generator(
                    training_set,
                    epochs=15,
                    validation_data=test_set,
                    validation_steps=10)

<ipython-input-6-f9915a5c690b>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  classifier.fit_generator(


Epoch 1/15
3/3 [==============================] - ETA: 0s - loss: 186.1616 - accuracy: 0.1458

3/3 [==============================] - 23s 6s/step - loss: 186.1616 - accuracy: 0.1458 - val_loss: 118.0500 - val_accuracy: 0.1667
Epoch 2/15
3/3 [==============================] - 2s 607ms/step - loss: 53.9615 - accuracy: 0.1562
Epoch 3/15
3/3 [==============================] - 1s 386ms/step - loss: 2.3033 - accuracy: 0.1979
Epoch 4/15
3/3 [==============================] - 1s 369ms/step - loss: 1.5466 - accuracy: 0.2500
Epoch 5/15
3/3 [==============================] - 1s 377ms/step - loss: 1.3835 - accuracy: 0.3438
Epoch 6/15
3/3 [==============================] - 1s 386ms/step - loss: 1.2855 - accuracy: 0.4271
Epoch 7/15
3/3 [==============================] - 1s 380ms/step - loss: 1.2372 - accuracy: 0.5104
Epoch 8/15
3/3 [==============================] - 2s 571ms/step - loss: 1.1500 - accuracy: 0.5312
Epoch 9/15
3/3 [==============================] - 2s 536ms/step - loss: 0.9552 - accuracy: 0.5833
Epoch 10/15
3/3 [==============================] - 1s 374ms/step - loss: 0.8404 - ac

#### Prediction

In [ ]:
import numpy as np
from tensorflow.keras.utils import load_img,img_to_array
import cv2
from keras.preprocessing import image

ImagePath='/content/drive/MyDrive/AI_and_ML_Face_Dataset/Keerthi/0.jpg'
test_image=load_img(ImagePath,target_size=(64, 64))
test_image=img_to_array(test_image)

cv2.imshow(test_image)

test_image=np.expand_dims(test_image,axis=0)

result=classifier.predict(test_image,verbose=0)

print('####'*10)
print('Prediction is: ',ResultMap[np.argmax(result)])

DisabledFunctionError: cv2.imshow() is disabled in Colab, because it causes Jupyter sessions
to crash; see https://github.com/jupyter/notebook/issues/3935.
As a substitution, consider using
  from google.colab.patches import cv2_imshow
